# Plant Pathology
*Identify the category of foliar diseases in apple trees.*

Folder structuur voor een snel en makkelijk overzicht:
```bash
.
├───datasets
│   └───train.csv
├───images
│   ├───resized_train_images
│   └───train_images
├───foliar_diseases.ipynb
└───image_resizer.ipynb
```


## 0. Modules importeren

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

import matplotlib.image as mpimg
from skimage.io import imread, imshow
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
from scipy import ndimage
#importeer Tensorflow namespaces
import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation
from keras.optimizers import SGD
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.callbacks import EarlyStopping
from keras import backend as K
from keras.utils import np_utils
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import cv2
import tensorflow as tf

## 1. Data inlezen.

### We maken een lijst van alle fotos in onze dataset.
\> lijst met lables  
\> lijst met fotos

In [2]:
directory = "images/resized_train_images/"
labels = pd.read_csv("datasets/train.csv")
label = labels["labels"].tolist()
images = labels["image"].tolist()
print(label[0], images[0])
img = io.imread(directory + images[0])/255
width = img.shape[0]
height = img.shape[1]

healthy 800113bb65efe69e.jpg


### LabelEncoder gebruiken om labels naar integers te converteren.

In [3]:
le = preprocessing.LabelEncoder()
le.fit(np.array(label).reshape(-1,))
y_encoded = le.transform(np.array(label).reshape(-1,))
print(y_encoded)

[ 3 11  9 ...  6 10  3]


### Fotos inlezen.

In [4]:
X = np.zeros((len(y_encoded),width,height,3)).astype(np.float32)

for i,img in enumerate(images):
    image = io.imread(directory + img)/255
    resized_image = image.astype(np.float32)
    X[i]=resized_image

### Train -en testsplits maken.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.33, random_state=1234)

print(X_train.shape)
print(y_train.shape)

(12483, 175, 175, 3)
(12483,)


### De uitkomst one-hot-encoderen.

In [6]:
y_train_onehot = to_categorical(y_train)

## 2. Transfer learning via VGG19

### VGG19 Model initialiseren

In [7]:
modelVGG19 = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(width,height,3))
type(modelVGG19)

tensorflow.python.keras.engine.functional.Functional

### VGG19 model omzetten naar sequentieel model.

In [8]:
# Store as a sequential model.

model = Sequential()

for layer in modelVGG19.layers[:]:
    model.add(layer)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 175, 175, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 175, 175, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 87, 87, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 87, 87, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 87, 87, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 43, 43, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 43, 43, 256)       2

### Layers in het model vastzetten.

In [9]:
for layer in model.layers:
    layer.trainable = False

### Top layers toevoegen aan het model

 - Flatten  
 - ReLU activation function  
 - Softmax activation function (12 outputs -> 11 plantenziekten + gezonde plant)

In [10]:
model.add(Flatten()) 
model.add(Dense(175, activation='relu'))
model.add(Dense(175, activation='relu'))
model.add(Dense(175, activation='relu'))
model.add(Dense(12, activation='softmax'))

### Model compileren en trainen

In [ ]:
#Compile
model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])
model.summary()

# Train
batch_size = 8
early_stopping =  EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train_onehot,batch_size=batch_size, epochs=100,
                    validation_split=0.2, callbacks=[early_stopping])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 175, 175, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 175, 175, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 87, 87, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 87, 87, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 87, 87, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 43, 43, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 43, 43, 256)       2

## 3. Uitvoeren op test set

In [ ]:
test_predict = model.predict_classes(X_test)
print(classification_report(y_test, test_predict))

## 4. Model opslaan

In [ ]:
model.save("bestModel.h5")